In [1]:
import geo

In [2]:
place_data = geo.get_place_data()

In [3]:
dups = place_data[place_data.index.duplicated()]
dups

,children,direct_children,name,type
key,,,,
E05014284,[],[],Hunmanby & Sherburn (part Scarborough),WD22


In [4]:
place_data[~place_data.index.duplicated()].to_json(f'../../src/_data/places.json', orient='index')